In [21]:
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np

In [22]:
import sys
import os

src_dir = os.path.join(os.getcwd(), 'src')
sys.path.append(os.path.abspath(src_dir))

from importlib import reload

import src.data_type.enum

reload(src.data_type.enum)

from src.data_type.enum import CityType

In [28]:
dataset_type = ["city", "train_station"]
selected_dataset = dataset_type[1]
brute_sentences_df = pd.read_csv('../data/dataset/test/brute_sentences_test3.csv')
df = pd.read_csv('../data/cities_and_train_stations.csv', sep=',')
df = df[df["class_name"] == selected_dataset]

In [29]:
def manage_intermediaries(sentence : str, row):
    intermediary_strings = ["INTERMEDIARY", "INTERMEDIARY.", "INTERMEDIARY,"]
    split_list = sentence.split(" ")
    current_intermediary = 0

    for idx, word in enumerate(split_list):
        if(word in intermediary_strings):

            random_label = df.iloc[np.random.randint(0, len(df))]["label"]
            while (random_label in sentence):
                random_label = df.iloc[np.random.randint(0, len(df))]["label"]

            split_list[idx] = random_label
            current_intermediary += 1
            row[f'intermediary_{current_intermediary}'] = random_label

    return " ".join(split_list)

In [30]:
def apply_location(sentence : str, new_df):
    row = {}

    for class_name in CityType._member_names_:
        if(class_name.upper() not in sentence):
            continue

        if(class_name == CityType.intermediary.value):
           sentence = manage_intermediaries(sentence, row) 
           continue

        sentence = sentence.replace(class_name.upper(), class_name)

        random_label = df.iloc[np.random.randint(0, len(df))]["label"]
        while (random_label in sentence):
            random_label = df.iloc[np.random.randint(0, len(df))]["label"]

        sentence = sentence.replace(class_name, random_label)
        row[class_name] = random_label

    row["sentence"] = sentence
    new_df.append(row)

In [31]:
clean_df = []
length = 2

with tqdm(total= length * len(brute_sentences_df), unit="sentence") as pbar:
    for idx, sentence in enumerate(brute_sentences_df["sentence"]):
        for i in range(length):
            apply_location(
                sentence,
                clean_df
            )
            pbar.update(1)

clean_df = pd.DataFrame(clean_df)

  0%|          | 0/150 [00:00<?, ?sentence/s]

In [32]:
clean_df.to_csv(f'../data/dataset/test/test3_{selected_dataset}.csv', index=False)